# Логичтическая регрессия, метод опорных векторов, one-hot кодирование

### О задании

В этом задании вы:
- настроите метод опорных векторов
- изучите методы работы с категориальными переменными

In [17]:
%pylab inline 
import pandas as pd  # тут импорт для работы с данными
from joblib import Parallel, delayed  # еще импорт для параллельного выполнения задач
import warnings, time  # эти модули для предупреждений и времени
warnings.simplefilter(action='ignore')  # игнорируем предупреждения
from sklearn.base import BaseEstimator  # это типа базовый класс для оценщиков
from sklearn.datasets import load_diabetes  # импорт функции для загрузки датасета
from sklearn.model_selection import train_test_split  # еще импорт для деления данных
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score  # функции для оценки качества моделей
import copy  # копирование объектов
import pandas as pd  # тут снова импорт pandas
from sklearn.model_selection import train_test_split  # опять импортируем функцию для деления данных
from sklearn.linear_model import LogisticRegression  # для логистической регрессии
from sklearn.preprocessing import OneHotEncoder  # кодирование категориальных признаков

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


__Задание 1.__ Обучение логистической регрессии на реальных данных и оценка качества классификации.

**(5 баллов)**


Загрузим данные с конкурса [Kaggle Porto Seguro’s Safe Driver Prediction](https://www.kaggle.com/c/porto-seguro-safe-driver-prediction) (вам нужна только обучающая выборка). Задача состоит в определении водителей, которые в ближайший год воспользуются своей автомобильной страховкой (бинарная классификация). Но для нас важна будет не сама задача, а только её данные. При этом под нужды задания мы немного модифицируем датасет.

In [18]:
data = pd.read_csv('train.csv', index_col=0)  # читаем файл 'train.csv' и делаем первый столбец индексом
target = data.target.values  # сохраняем значения столбца 'target' как массив целевых переменных
data = data.drop('target', axis=1)  # удаляем столбец 'target', axis=1 для удаления по столбцам

Пересемплируем выборку так, чтобы положительных и отрицательных объектов в выборке было одинаковое число. Разделим на обучающую и тестовую выборки.

In [19]:
np.random.seed(910)  # ну типа старт генерации случайных чисел
mask_plus = np.random.choice(np.where(target == 1)[0], 100000, replace=True)  # выбор случайных значений, когда цель это 1
mask_zero = np.random.choice(np.where(target == 0)[0], 100000, replace=True)  # выбор случайных значений, когда цель это 0
mask = np.concatenate([mask_plus, mask_zero])  # ага, объединяем всё это в один массив
mask = np.sort(mask)  # ну и отсортировали его, чтоб было аккуратно

data = data.iloc[mask]  # применяем маску к данным, получается вот такой фильтр
target = target[mask]  # а это уже фильтр для целевых значений

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.5, random_state=73)  # ну и делим всё это на обучающие и тестовые наборы


Не забудьте отнормировать признаки (можно воспользоваться StandardScaler или сделать это вручную). Пока не будем обращать внимание на то, что некоторые признаки категориальные (этим мы займёмся позже).

In [20]:
def normalize(data):
    new_data = copy.deepcopy(data)  # делаем копию данных, чтобы не изменять оригинальные
    for c in data.columns:  # проходим по всем столбцам в данных
        new_data[c] = (new_data[c] - new_data[c].min()) / (new_data[c].max() - new_data[c].min())  # нормализуем значения столбца
    return new_data  # возвращаем нормализованные данные


Обучите логистическую регрессию с удобными для вас параметрами, примените регуляризацию. Сделайте предсказание на тестовой части выборки. Посчитайте accuracy, precision, recall и F меру

In [21]:
def get(X_train, X_test, y_train, y_test):  # объявляем функцию get с параметрами X_train, X_test, y_train, y_test
    st = time.time()  # запоминаем текущее время
    model = LogisticRegression(max_iter=1000)  # создаем модель логистической регрессии с максимальным количеством итераций 1000
    model.fit(X_train, y_train)  # обучаем модель на обучающих данных
    y_pred = model.predict(X_test)  # делаем прогноз на тестовых данных
    accuracy = accuracy_score(y_test, y_pred)  # вычисляем точность прогноза
    precision = precision_score(y_test, y_pred)  # вычисляем точность
    recall = recall_score(y_test, y_pred)  # вычисляем полноту
    f1 = f1_score(y_test, y_pred)  # вычисляем F1-меру
    print(f"Accuracy:", accuracy)  # выводим точность
    print(f"Precision:", precision)  # выводим точность
    print(f"Recall:", recall)  # выводим полноту
    print(f"F1 Score:", f1)  # выводим F1-меру
    en = time.time()  # запоминаем текущее время
    print(f"Time taken: {en - st} s")  # выводим время выполнения функции


get(*train_test_split(normalize(data), target, test_size=0.5, random_state=73))  # вызываем функцию get с аргументами, полученными из train_test_split и normalize


Accuracy: 0.5885
Precision: 0.5971422338568936
Recall: 0.5468475307655426
F1 Score: 0.5708892967381329
Time taken: 3.6531147956848145 s


__Выводы__ в свободной форме:

## Часть 2. Работа с категориальными переменными

В этой части мы научимся обрабатывать категориальные переменные, так как закодировать их в виде чисел недостаточно (это задаёт некоторый порядок, которого на категориальных переменных может и не быть). Существует два основных способа обработки категориальных значений:
- One-hot-кодирование
- Счётчики (CTR, mean-target кодирование, ...) — каждый категориальный признак заменяется на среднее значение целевой переменной по всем объектам, имеющим одинаковое значение в этом признаке.

Начнём с one-hot-кодирования. Допустим наш категориальный признак $f_j(x)$ принимает значения из множества $C=\{c_1, \dots, c_m\}$. Заменим его на $m$ бинарных признаков $b_1(x), \dots, b_m(x)$, каждый из которых является индикатором одного из возможных категориальных значений:
$$
b_i(x) = [f_j(x) = c_i]
$$

__Задание 1.__ Закодируйте все категориальные признаки с помощью one-hot-кодирования. Обучите логистическую регрессию и посмотрите, как изменилось качество модели (с тем, что было ранее). Измерьте время, потребовавшееся на обучение модели.

__(3 балла)__

In [22]:
data.describe()  # вот тут мы выводим статистическое описание данных, ну так, чтобы понять, что в них происходит

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,...,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000
mean,1.998215,1.366550,4.483870,0.429490,0.502265,0.351100,0.295375,0.175920,0.177605,0.000470,...,5.443865,1.443745,2.873590,7.544455,0.123355,0.630875,0.553405,0.287530,0.345000,0.152800
std,2.017199,0.674421,2.739255,0.496689,1.501934,0.477315,0.456212,0.380753,0.382181,0.021674,...,2.342462,1.201163,1.692875,2.745287,0.328845,0.482569,0.497141,0.452612,0.475369,0.359796
min,0.000000,-1.000000,0.000000,-1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,4.000000,1.000000,2.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,1.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,5.000000,1.000000,3.000000,7.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
75%,3.000000,2.000000,7.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,...,7.000000,2.000000,4.000000,9.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
max,7.000000,4.000000,11.000000,1.000000,6.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,18.000000,8.000000,13.000000,22.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


Как можно было заменить, one-hot-кодирование может сильно увеличивать количество признаков в датасете, что сказывается на памяти, особенно, если некоторый признак имеет большое количество значений. Эту проблему решает другой способ кодирование категориальных признаков — счётчики. Основная идея в том, что нам важны не сами категории, а значения целевой переменной, которые имеют объекты этой категории. Каждый категориальный признак мы заменим средним значением целевой переменной по всем объектам этой же категории:
$$
g_j(x, X) = \frac{\sum_{i=1}^{l} [f_j(x) = f_j(x_i)][y_i = +1]}{\sum_{i=1}^{l} [f_j(x) = f_j(x_i)]}
$$

__Задание 2.__ Закодируйте категориальные переменные с помощью счётчиков (ровно так, как описано выше без каких-либо хитростей). Обучите логистическую регрессию и посмотрите на качество модели на тестовом множестве. Сравните время обучения с предыдущим экспериментов. Заметили ли вы что-то интересное?

__(2 балла)__

In [23]:
categorical_features = [c for c in data.columns if c.endswith('_cat')]  # выбираем категориальные признаки, которые заканчиваются на '_cat'

encoder = OneHotEncoder(sparse_output=False, drop="first")  # создаем кодировщик OneHotEncoder с параметрами sparse_output=False и drop="first"
encoded_features = encoder.fit_transform(data[categorical_features])  # кодируем категориальные признаки

data_one_hot = data.drop(categorical_features, axis=1)  # создаем датафрейм без категориальных признаков
encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_features))  # создаем датафрейм с закодированными признаками

data_one_hot.reset_index(inplace=True, drop=True)  # сбрасываем индексы
data_one_hot = pd.concat([data_one_hot, encoded_df], axis=1)  # объединяем исходные данные с закодированными признаками

get(*train_test_split(normalize(data_one_hot), target, test_size=0.5, random_state=73))  # запускаем функцию get с аргументами, полученными из train_test_split и normalize

Accuracy: 0.59355
Precision: 0.6014488066748593
Recall: 0.5573158062969474
F1 Score: 0.578541876211906
Time taken: 24.93733859062195 s


__Вывод:__

Отметим, что такие признаки сами по себе являются классификаторами и, обучаясь на них, мы допускаем "утечку" целевой переменной в признаки. Это ведёт к переобучению, поэтому считать такие признаки необходимо таким образом, чтобы при вычислении для конкретного объекта его целевая метка не использовалась. Это можно делать следующими способами:
- вычислять значение счётчика по всем объектам расположенным выше в датасете (например, если у нас выборка отсортирована по времени)
- вычислять по фолдам, то есть делить выборку на некоторое количество частей и подсчитывать значение признаков по всем фолдам кроме текущего (как делается в кросс-валидации)
- внесение некоторого шума в посчитанные признаки (необходимо соблюсти баланс между избавление от переобучения и полезностью признаков).

__Задание 3.__ Реализуйте корректное вычисление счётчиков двумя из трех вышеперчисленных способов, сравните. Снова обучите логистическую регрессию, оцените качество. Сделайте выводы.

__(3 балла)__

In [24]:
# по объектам расположенным выше в датасете
X = data.copy()  # создаем копию данных и целевой переменной
y = target.copy()  # создаем копию целевой переменной
for feature in categorical_features:  # проходим по всем категориальным признакам
    ind = 0  # начальный индекс
    ans = []  # список для хранения ответов
    mp = {}  # словарь для хранения сумм и количеств значений признака
    for j in X[feature].tolist():  # проходим по значениям признака
        mp[j] = mp.get(j, [0, 0])  # добавляем значение в словарь, если его там нет
        mp[j][0] += y[ind]  # добавляем к сумме значение целевой переменной
        mp[j][1] += 1  # увеличиваем счетчик количества появлений значения
        ans.append(mp[j][0] / mp[j][1])  # вычисляем среднее значение для значения признака
        ind += 1  # увеличиваем индекс
    X[feature] = pd.Series(ans, index=X[feature].index)  # заменяем значения признака на средние

%time get(*train_test_split(normalize(X), y, test_size=0.5, random_state=73))  # запускаем функцию get с аргументами, полученными из train_test_split и normalize, и замеряем время выполнения

Accuracy: 0.59549
Precision: 0.6020527423022164
Recall: 0.5660060731980182
F1 Score: 0.5834732018740668
Time taken: 11.993443012237549 s
CPU times: total: 12.7 s
Wall time: 12.8 s


In [ ]:
# по фолдам
X = data.copy()  # создаем копию данных и целевой переменной
y = target.copy()  # создаем копию целевой переменной
B = int(len(X) ** 0.5)  # определяем количество фолдов

for feature in categorical_features:  # проходим по всем категориальным признакам
    lst = X[feature].to_list()  # преобразуем значения признака в список
    cnts = [{} for j in range(len(X) // B + 2)]  # список словарей для подсчета количества значений в каждом фолде
    sums = [{} for j in range(len(X) // B + 2)]  # список словарей для подсчета суммы целевой переменной в каждом фолде

    for i in range(len(X)):  # проходим по всем объектам
        if len(cnts[i // B]) == 0:  # если текущий фолд пустой
            cnts[i // B + 1] = cnts[i // B].copy()  # копируем предыдущий фолд
            sums[i // B + 1] = sums[i // B].copy()  # копируем предыдущий фолд
        cnts[i // B + 1][lst[i]] = cnts[i // B + 1].get(lst[i], 0) + 1  # увеличиваем счетчик для текущего значения
        sums[i // B + 1][lst[i]] = sums[i // B + 1].get(lst[i], 0) + y[i]  # добавляем значение целевой переменной для текущего значения

    ind = 0  # начальный индекс
    ans = []  # список для хранения ответов
    for j in X[feature].tolist():  # проходим по значениям признака
        sum = sums[ind // B].get(j, 0) + sums[-1].get(j, 0) - sums[ind // B + 1].get(j, 0)  # сумма целевой переменной для текущего значения
        cnt = cnts[ind // B].get(j, 0) + cnts[-1].get(j, 0) - cnts[ind // B + 1].get(j, 0)  # количество значений для текущего значения
        ans.append(0 if cnt == 0 else sum / cnt)  # вычисляем среднее значение для значения признака
        ind += 1  # увеличиваем индекс

    X[feature] = pd.Series(ans, index=X[feature].index)  # заменяем значения признака на средние

%time get(*train_test_split(normalize(X), y, test_size=0.5, random_state=73))  # запускаем функцию get с аргументами, полученными из train_test_split и normalize, и замеряем время выполнения


__Вывод:__

## Часть 2. Метод опорных векторов и калибровка вероятностней

__Задание 1.__ Обучение и применение метода опорных векторов.

__(1 балл)__

Обучите метод опорных векторов (воспользуйтесь готовой реализацией LinearSVC из sklearn). Используйте уже загруженные и обработанные в предыдущей части данные.

In [14]:
from sklearn.svm import LinearSVC  # импортируем класс LinearSVC из модуля sklearn.svm

X_train, X_test, y_train, y_test = train_test_split(normalize(X), y, test_size=0.5, random_state=73)  # разделяем данные на обучающий и тестовый наборы

svc = LinearSVC(random_state=73)  # создаем модель LinearSVC с заданным random_state

svc.fit(X_train, y_train)  # обучаем модель на обучающих данных

y_pred = svc.predict(X_test)  # делаем прогноз на тестовых данных

На той же тестовой части посчитайте все те же метрики. Что вы можете сказать о полученных результатах?

In [15]:
accuracy = accuracy_score(y_test, y_pred)  # вычисляем точность прогноза
precision = precision_score(y_test, y_pred)  # вычисляем точность
recall = recall_score(y_test, y_pred)  # вычисляем полноту
f1 = f1_score(y_test, y_pred)  # вычисляем F1-меру

print(f"Accuracy:", accuracy)  # выводим точность
print(f"Precision:", precision)  # выводим точность
print(f"Recall:", recall)  # выводим полноту
print(f"F1 Score:", f1)  # выводим F1-меру


Accuracy: 0.60508
Precision: 0.6122444642781248
Recall: 0.575575355601726
F1 Score: 0.5933439051012211


__Вывод:__